In [1]:
#pip install scikit-learn
#pip install pandas

# Main imports pandas and sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# We may need to do some web scraping to get the review text
import urllib.request
from bs4 import BeautifulSoup

#Other imports
import time
import sys
from os import system
from IPython.display import clear_output

# Load in data
### There are a few steps to getting clean data
- Read data from kaggle into dataframes
- Get rid of unecessary features of the data
- (For game spot data) get the review text from the web
- Clean data using dropna

## Read in data from csv

In [2]:
# download dataset at: https://www.kaggle.com/datasets/joyshil0599/multi-decade-video-game-review-dataset
GAME_SPOT_DATASET_PATH = "data/game_spot_data.csv"
# downoad dataset at: https://www.kaggle.com/datasets/andrewmvd/steam-reviews
STEAM_DATASET_PATH = 'data/steam_data.csv'
# download dataset at: https://www.kaggle.com/datasets/noahx1/elden-ring-steam-reviews
ELDEN_RING_DATASET_PATH = "data/elden_ring_data.csv"

# Read in the datasets
game_spot_data = pd.read_csv(GAME_SPOT_DATASET_PATH)
steam_data = pd.read_csv(STEAM_DATASET_PATH)
elden_ring_data = pd.read_csv(ELDEN_RING_DATASET_PATH)

In [3]:
# Lets only keep the columns we want
game_spot_cols = ['Review link','Rating/10']
if('Review' in game_spot_data.columns):
    game_spot_cols.append('Review')
steam_cols = ['review_text','review_score']
elden_ring_cols = ['voted_up','review']

game_spot_data = game_spot_data[game_spot_cols]
steam_data = steam_data[steam_cols]
elden_ring_data = elden_ring_data[elden_ring_cols]

# Lets rename some columns and change them to be consistant
steam_data['review_score'].replace({1:True,-1:False},inplace=True)
steam_data.rename(columns={'review_score':'up_vote','review_text':'Review'}, inplace=True)
elden_ring_data.rename(columns={'voted_up':'up_vote','review':'Review'},inplace=True)

# Read in reviews from web (for game spot data)

In [4]:
# Now lets define how to get the text of the reviews from the web
from multiprocessing.dummy import Pool as ThreadPool
def ScrapeSingle(link): 
    system('cls')
    print(time.time())
    try:
#        print("Scraping Single")
        page = urllib.request.urlopen(link).read()
        page = BeautifulSoup(page)
        review = ""
        body = page.find(class_="article-body typography-format")
        paragraphs = body.find_all("p")
        if(len(paragraphs)==0):
            raise Exception('NO REVIEW TEXT FOUND')
        for p in paragraphs:
            review+=p.text+" "
        return review
    except:
#        print("Scraping Single")
        return None

        
def GetReviewsFromWebParallel(num_threads=2048):
    print("In Parallel")
    num_processed = 1
    prev_time = time.time()
    check_interval = 5    
    num_links = len(game_spot_data.index)
    print("Making pool")
    pool = ThreadPool(num_threads)
    print("Mapping Pool")
    results = pool.map(ScrapeSingle, game_spot_data['Review link'])
    print("Closing Pool")
    pool.close()
    print("Joining Pool")
    pool.join()
    print("Pool joined.")
    reviews = []
    failed_links = []
    print("Entering for loop with zip")
    for r, link in zip(results, game_spot_data['Review link']):
               # Display progress
        num_processed = num_processed + 1
        if(num_processed % check_interval == 0):
            prediction = (time.time() - prev_time)/(check_interval) * (num_links-len(reviews))
            prev_time = time.time()
            system('cls')
            clear_output(wait=True)
            print(f"Reviews lost: {len(failed_links)}")
            print(f"processed: {num_processed} / {num_links}")
            print(f"Time left: {int(prediction)} s")  
        if r is None:
            print("No review")
            reviews.append(None)
            failed_links.append(link)
        else:
            print("Appending review")
            reviews.append(r)  
    game_spot_data['Review'] = reviews
        
def GetReviewsFromWeb():
    num_processed = 1
    prev_time = time.time()
    num_links = len(game_spot_data.index)
    check_interval = 5
    reviews = []
    failed_links = []
    for link in game_spot_data['Review link']:
        # Try to get the review text
        try:
            page = urllib.request.urlopen(link).read()
            page = BeautifulSoup(page)
            review = ""
            body = page.find(class_="article-body typography-format")
            paragraphs = body.find_all("p")
            if(len(paragraphs)==0):
                raise Exception('NO REVIEW TEXT FOUND')
            for p in paragraphs:
                review+=p.text+" "
            reviews.append(review)
        # If we could not get the review text
        except:
            reviews.append(None)
            failed_links.append(link)
        # Display progress
        num_processed = num_processed + 1
        if(num_processed % check_interval == 0):
            prediction = (time.time() - prev_time)/(check_interval) * (num_links-len(reviews))
            prev_time = time.time()
            system('cls')
            clear_output(wait=True)
            print(f"Reviews lost: {len(failed_links)}")
            print(f"processed: {num_processed} / {num_links}")
            print(f"Time left: {int(prediction)} s")  
    # Update dataframe and write to file
    game_spot_data['Review'] = reviews
    

In [ ]:
# If gamespot data does not have review then we need to get it from the web
if(not 'Review' in game_spot_data.columns):
    print("No reviews. Calling Parallel")
    GetReviewsFromWebParallel()
    print("Called Parallel")
    game_spot_data.to_csv(GAME_SPOT_DATASET_PATH, index=False)

No reviews. Calling Parallel
In Parallel
Making pool
Mapping Pool
1681848333.0258036
1681848333.0563226
1681848333.0723262
1681848333.073326
1681848333.073326
1681848333.073326
1681848333.074326
1681848333.074326
1681848333.074326
1681848333.074326
1681848333.074326
1681848333.074326
1681848333.074326
1681848333.0753262
1681848333.0753262
1681848333.09833171681848333.1143365
1681848333.1153352
1681848333.130339
1681848333.150854
1681848333.1959286

1681848333.2536197
1681848333.3195176
1681848333.365039
1681848333.365039
1681848333.365039
1681848333.389045
1681848333.4645758
1681848333.5065851
1681848333.5257957
1681848333.5257957
1681848333.586203
1681848333.8133957
1681848334.2196276
1681848337.0764549
1681848337.124015
1681848337.1545298
1681848337.1555302
1681848337.1555302
1681848337.1658435
1681848337.1765747
1681848337.1765747
1681848337.1869757
1681848337.2128477
1681848337.2325175
1681848337.2973528
1681848337.2973528
1681848337.3266294
1681848337.3266294
1681848337.3564043
16

1681848352.96446731681848352.9768336
1681848353.0139048
1681848353.0139048
1681848353.0219684
1681848353.0219684
1681848353.043145
1681848353.043145
1681848353.043145
1681848353.0642915
1681848353.0642915
1681848353.0642915
1681848353.076281
1681848353.0843804
1681848353.0843804
1681848353.1042922
1681848353.1255007
1681848353.1494691
1681848353.1494691
1681848353.1699245
1681848353.1779702
1681848353.1779702
1681848353.1779702
1681848353.1779702
1681848353.2066286
1681848353.2066286
1681848353.2384067
1681848353.2384067
1681848353.2384067
1681848353.2626176
1681848353.2626176
1681848353.2626176
1681848353.2888155
1681848353.2888155
1681848353.3129554
1681848353.3129554
1681848353.332689
1681848353.349724
1681848353.3669095
1681848353.4121485
1681848353.4121485
1681848353.4216118
1681848353.4216118
1681848353.4216118
1681848353.42874
1681848353.46864
1681848353.46864
1681848353.5301516
1681848353.5301516
1681848353.5551593
1681848353.5621614
1681848353.583805
1681848353.583805
16818483

1681848366.81906751681848366.8204567
1681848366.843587
1681848366.843587
1681848366.8946424
1681848366.912294
1681848366.9867895
1681848366.9981682
1681848367.0181792
1681848367.0631225
1681848367.0631225
1681848367.1426978
1681848367.1856704
1681848367.23547
1681848367.2454886
1681848367.2454886
1681848367.2454886
1681848367.2464888
1681848367.2629888
1681848367.2639925
1681848367.2639925
1681848367.3355443
1681848367.3355443
1681848367.3506765
1681848367.3506765
1681848367.392189
1681848367.4297316
1681848367.437248
1681848367.4678562
1681848367.4698713
1681848367.492169
1681848367.5021996
1681848367.5021996
1681848367.5409696
1681848367.55762
1681848367.55762
1681848367.6108673
1681848367.6261513
1681848367.6261513
1681848367.6500018
1681848367.6500018
1681848367.7129567
1681848367.7139578
1681848367.7566702
1681848367.7566702
1681848367.7566702
1681848367.8143947
1681848367.8325706
1681848367.8668146
1681848367.8668146
1681848367.9077091
1681848367.9537199
1681848367.9717245
168184

1681848623.0154412
1681848628.70120221681848629.27803421681848629.3259265

1681848629.6326647

1681848630.3173928
1681848630.347498
1681848630.411252
1681848637.3728747
1681848638.71528671681848638.71638

1681848638.8565395
1681848638.8575425
1681848638.9187212
1681848639.1235433
1681848639.2005308
1681848639.242244
1681848639.311462
1681848639.5169988
1681848639.7007942
1681848640.265659
1681848640.2959025
1681848648.8957205
1681848651.665949
1681848702.7592149
1681848706.2217581
1681848711.0302439
1681848711.2651634
1681848718.8905225
1681848718.9851758
1681848719.10877
1681848719.2984734
1681848719.5180109
1681848719.5810504
1681848719.5810504
1681848719.6585505
1681848719.9860811
1681848720.126841
1681848720.1756403
1681848720.1756403
1681848720.1756403
1681848720.2353182
1681848720.3622823
1681848720.517827
1681848721.0340629
1681848721.1110687
1681848721.11334
1681848721.1741211
1681848721.1908662
1681848721.6583865
1681848721.9062974
1681848721.9785807
1681848722.217597
16818487

1681848965.93086961681848965.9647748
1681848966.7502925

1681848968.5889804
1681848969.6830134
1681848969.7792482
1681848969.9497323
1681848970.0502422
1681848970.0502422
1681848970.0771108
1681848970.1255257
1681848970.5466373
1681848970.6753824
1681848971.1636577
1681848971.2870586
1681848971.3385525
1681848971.385525
1681848971.4016852
1681848971.621419
1681848971.6684637
1681848971.8244748
1681848971.8560967
1681848971.9486506
1681848971.950181
1681848972.1200442
1681848972.2771544
1681848972.3568077
1681848972.3817463
1681848972.4031112
1681848972.5286262
1681848972.701233
1681848972.7505665
1681848972.8748608
1681848972.906282
1681848973.0148592
1681848973.3448043
1681848974.20934081681848974.3339534
1681848974.3499277
1681848974.350979
1681848974.5664802
1681848974.625333
1681848974.625333

1681848975.073202
1681848975.6962
1681848976.3109875
1681848978.9230132
1681848979.78054261681848980.2024338

1681848980.3587346
1681848980.5767725
1681848980.5921452
1681848980.9560037
16818

...

1681851659.67168971681851659.7244148

1681851660.0885413
1681851660.2577376
1681851661.253042
1681851661.7248645
1681851662.300546
1681851662.7552123
1681851663.2118192
1681851663.4250526
1681851663.613418
1681851663.7069275
1681851664.281467
1681851664.9706964
1681851665.0022395
1681851665.0032399
1681851665.8747537
1681851666.3224635
1681851666.98329351681851667.1852648

1681851667.2952237
1681851667.6791995
1681851668.0932806
1681851668.4116325
1681851668.8022358
1681851669.0241969
1681851669.2510877
1681851669.468527
1681851669.5267909
1681851670.37134841681851670.5113199
1681851670.8617613

1681851671.2502038
1681851671.2712228
1681851671.9448948
1681851672.9894323
1681851673.5774372
1681851673.6594603
1681851673.8573778
1681851673.9589484
1681851674.7453391681851674.9355874

1681851674.9551136
1681851675.3124993
1681851675.6454546
1681851675.88705
1681851675.8906074
1681851676.37168
1681851676.61848
1681851677.0981352
1681851677.6020684
1681851677.98274251681851678.0794992
168185

1681851794.66446421681851794.7891057
1681851794.835369
1681851794.85328
1681851794.8700829
1681851795.2204924
1681851795.2544394

1681851795.647103
1681851795.8202586
1681851795.8836472
1681851796.654928
1681851796.700345
1681851797.0664322
1681851797.0803556
1681851797.8740668
1681851798.5596828
1681851798.87414311681851798.8767009

1681851799.1242092
1681851799.3160212
1681851799.4619358
1681851799.5682023
1681851802.52635861681851803.128167
1681851803.362986
1681851803.4275303

1681851803.6301205
1681851803.775285
1681851803.8558211
1681851803.8849363
1681851804.245864
1681851804.3310935
1681851804.4522314
1681851804.4978404
1681851804.5610015
1681851804.5939062
1681851804.655007
1681851804.8530378
1681851804.8540587
1681851804.8890777
1681851804.9199774
1681851808.2448628
1681851808.3878498
1681851808.440991
1681851808.4920785
1681851808.6685503
1681851808.7150035
1681851808.9265084
1681851808.9371555
1681851808.9371555
1681851808.9565916
1681851809.3203337
1681851809.3331954
16818

1681851942.9654486
1681851943.2256382
1681851943.327471
1681851943.7039602
1681851944.4690626
1681851944.8913748
1681851944.9544988
1681851945.0962994
1681851945.1783524
1681851945.3405044
1681851945.5933306
1681851945.8765357
1681851945.956375
1681851946.06547
1681851946.20946
1681851946.5897932
1681851947.334334
1681851947.4855504
1681851947.6741889
1681851948.4131784
1681851948.6193426
1681851948.6814673
1681851948.7239332
1681851948.7417216
1681851949.1824586
1681851953.71809581681851954.3642197
1681851954.4071076
1681851955.0239818
1681851955.0422137
1681851955.107759
1681851955.1185443
1681851955.1374092
1681851955.2450762
1681851955.3407376
1681851955.4248693
1681851955.4521534
1681851955.5328412

1681851956.191047
1681851956.5445395
1681851957.3655553
1681851957.4604485
1681851958.779825
1681851960.9617317
1681851961.0877726
1681851961.3935864
1681851962.0264804
1681851962.6393383
1681851963.0280745
1681851963.0833244
1681851963.6357079
1681851963.910977
1681851963.9872458
1681

1681852103.55399681681852103.6405194
1681852103.7187364

1681852104.1184623
1681852104.5860584
1681852104.8200557
1681852104.8530216
1681852104.9945743
1681852105.0599134
1681852105.0971403
1681852105.151675
1681852105.1526754
1681852105.2360039
1681852105.425448
1681852105.4276981
1681852105.4618669
1681852105.5211396
1681852105.87764
1681852105.9421928
1681852106.0082898
1681852106.0521395
1681852106.1718981
1681852106.1787827
1681852106.2233148
1681852106.240381
1681852106.2556639
1681852106.3798492
1681852106.632298
1681852106.7124856
1681852106.9471717
1681852107.7999621681852107.8285594

1681852108.8083112
1681852110.21479871681852110.2629075

1681852111.0664687
1681852111.5569942
1681852112.1368
1681852112.2452207
1681852112.5596936
1681852113.4292524
1681852113.6348224
1681852114.85705691681852114.9077973
1681852114.966961
1681852115.1729856

1681852115.2729719
1681852115.2739947
1681852115.3351972
1681852115.411077
1681852115.5196037
1681852115.7763588
1681852115.8283455
16818

1681852348.6925836
1681852354.5806108
1681852354.779063
1681852355.89302951681852356.067995
1681852356.4132173
1681852356.490165
1681852356.5838392

1681852357.2002137
1681852357.2906463
1681852357.3719923
1681852357.8781695
1681852359.1834192
1681852359.4047794
1681852360.4680605
1681852361.0972118
1681852361.9720922
1681852362.2768552
1681852362.7898784
1681852364.3177826
1681852364.4451199
1681852364.598661
1681852364.6012225
1681852364.7486696
1681852366.4760431681852366.680827
1681852366.6997354

1681852366.9643083
1681852367.0609484
1681852367.1440737
1681852367.4918168
1681852367.5055413
1681852367.7791238
1681852367.89637
1681852367.995899
1681852368.2163334
1681852368.658652
1681852369.3163911681852369.6881502
1681852369.861688

1681852371.317209
1681852371.7373557
1681852373.7760875
1681852374.4704306
1681852374.9966671
1681852376.3699396
1681852376.4112573
1681852376.5143733
1681852376.836722
1681852376.9078796
1681852376.9118805
1681852377.0535283
1681852377.3223975
1681852

# Now lets define our classifications (Sentiment)

In [ ]:
# We define 2 sentiments to classify
GOOD = 'Good'
BAD = 'Bad'

In [ ]:
def GetSentimentFromRating(rating):
    if(rating is None):
        return None
    if(rating > 7):
        return GOOD
    if(rating > 0):
        return BAD

In [ ]:
def GetSentimentFromUpVote(up_vote):
    if(up_vote):
        return GOOD
    else:
        return BAD

In [ ]:
def AddSentiment(df):
    s = []
    if('Rating/10' in df.columns):
        for rating in game_spot_data['Rating/10'].to_list():
            s.append(GetSentimentFromRating(rating))
    elif('up_vote' in df.columns):
        for upvote in df['up_vote'].to_list():
            s.append(GetSentimentFromUpVote(upvote))
    else:
        raise Exception(f"No column in dataframe to use for sentiment! {df.columns}")
    df['Sentiment'] = s
    print("DATA DISTRIBUTION:")
    SENTIMENTS = df['Sentiment'].unique()
    for s in SENTIMENTS:
        print(f"{s}: {sum(1 for i in df['Sentiment'] if i == s)}")

In [ ]:
AddSentiment(game_spot_data)
AddSentiment(steam_data)
AddSentiment(elden_ring_data)

# Clean data

In [ ]:
def CleanData(df):
    # Now lets finish cleaning the data by dropping any invalid rows
    old_size = df.shape
    df.dropna(inplace=True)
    new_size = df.shape
    print("Dropping rows")
    print(f"Old count: {old_size[0]}")
    print(f"New count: {new_size[0]}")
    print(f"Removed {old_size[0]-new_size[0]} rows.")
    

In [ ]:
CleanData(game_spot_data)
CleanData(steam_data)
CleanData(elden_ring_data)

# Now we split our data

In [ ]:
def SplitData(df,test_split=0.3):
    #split datasets
    X = df['Review']
    y = df['Sentiment'] 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_split, random_state=1) 
    X_train.to_list()
    X_test = X_test.to_list()
    y_train = y_train.to_list()
    y_test = y_test.to_list()
    return X_train, X_test, y_train, y_test 

In [ ]:
game_spot_X_train, game_spot_X_test, game_spot_y_train, game_spot_y_test = SplitData(game_spot_data)
steam_X_train, steam_X_test, steam_y_train, steam_y_test = SplitData(steam_data)
elden_ring_X_train, elden_ring_X_test, elden_ring_y_train, elden_ring_y_test = SplitData(elden_ring_data)

## Vectorize the data
We will be using a method called **Term Frequency and Inverse Document Frequency (TF-IDF)**. 
Here is how it works: https://medium.com/@vasista/preparing-the-text-data-with-scikit-learn-b31a3df567e

#### sklearn supplies an easy way to implement this

In [ ]:
def CreateVectorizer(X_train):
    vectorizer = TfidfVectorizer(min_df = 5,max_df = 0.8,sublinear_tf = True,use_idf = True)
    train_vectors = vectorizer.fit_transform(X_train)
    return vectorizer, train_vectors

# Creating svm
#### Helpful link to understand how this works:
- https://medium.com/@vasista/sentiment-analysis-using-svm-338d418e3ff1



In [ ]:
def TestSVM(svm_linear, vectorizer, X_test, y_test): 
    print('Testing SVM ...')
    start_time = time.time()
    test_vectors = vectorizer.transform(X_test)
    prediction = svm_linear.predict(test_vectors)
    end_time = time.time()
    print(f"Tested SVM in {end_time-start_time} s")
    report = classification_report(y_test, prediction, output_dict=True, zero_division=0)
    cm = confusion_matrix(y_test, prediction)
    cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels = svm_linear.classes_)
    cm_disp.plot()
    for key in report.keys():
        print(f'{key}: {report[key]}\n')
    return report

In [ ]:
def CreateSVM(x_train, y_train):
    print('training SVM...')
    start_time = time.time()
    vectorizer, train_vectors = CreateVectorizer(x_train)
    svm_linear = svm.LinearSVC()
    start_time = time.time()
    svm_linear.fit(train_vectors, y_train)
    end_time = time.time()
    print(f"Trained SVM in {end_time-start_time} s")
    return svm_linear, vectorizer

# Train SVM with game spot data

In [ ]:
game_spot_svm, game_spot_vectorizer = CreateSVM(game_spot_X_train, game_spot_y_train)

### Test game_spot_svm on game_spot test data

In [ ]:
report = TestSVM(game_spot_svm, game_spot_vectorizer, game_spot_X_test, game_spot_y_test)

### Test game_spot_svm on steam data

In [ ]:
report = TestSVM(game_spot_svm, game_spot_vectorizer, steam_X_test, steam_y_test)

### Test game_spot_svm on elden ring data

In [ ]:
report = TestSVM(game_spot_svm, game_spot_vectorizer, elden_ring_X_test, elden_ring_y_test)

# Train SVM with elden ring data

In [ ]:
elden_ring_svm, elden_ring_vectorizer = CreateSVM(elden_ring_X_train, elden_ring_y_train)

### Test elden_ring_svm on elden ring test data

In [ ]:
report = TestSVM(elden_ring_svm, elden_ring_vectorizer, elden_ring_X_test, elden_ring_y_test)

### Test elden_ring_svm on steam data

In [ ]:
report = TestSVM(elden_ring_svm, elden_ring_vectorizer, steam_X_test, steam_y_test)

### Test elden_ring_svm on game spot data

In [ ]:
report = TestSVM(elden_ring_svm, elden_ring_vectorizer, game_spot_X_test, game_spot_y_test)

# Train SVM with Steam Data

In [ ]:
steam_svm, steam_vectorizer = CreateSVM(steam_X_train, steam_y_train)

### Test steam_svm on steam data

In [ ]:
report = TestSVM(steam_svm, steam_vectorizer, steam_X_test, steam_y_test)

### Test steam_svm on game spot data

In [ ]:
report = TestSVM(steam_svm, steam_vectorizer, game_spot_X_test, game_spot_y_test)

### Test steam_svm on elden ring data

In [ ]:
report = TestSVM(steam_svm, steam_vectorizer, elden_ring_X_test, elden_ring_y_test)